In [1]:
import chromadb
import pandas as pd

from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentType
from langchain.agents import Tool, initialize_agent

import openai
import requests
import openai
import requests

from textblob import TextBlob  # Para análisis de sentimientos

print(chromadb.__version__)

client = chromadb.PersistentClient(path=".")
collection = client.get_or_create_collection("hidrogeno_verde")
df = pd.read_csv('hidrogeno_verde.csv')

0.5.23


In [2]:
# Convertir la columna 'date' al tipo datetime
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
# Formatear la columna 'date' al formato YYYY-MM-DD
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [3]:
df['combined'] = df['title'] + ' ' + df['text']

In [4]:
# Añadir noticias a ChromaDB sin proporcionar los embeddings
collection.upsert(
    documents=df['combined'].tolist(),
        metadatas=[{'media_outlet': row['media_outlet'], 'date': row['date'], 'title': row['title'], 'url': row['url']} for _, row in df.iterrows()],
    ids=[f'noticia_{i}' for i in range(len(df))]
)
### cuidado con la metadata

In [5]:
query = "¿Cuales son las personas que más se mencionan entre dos fechas sobre la temática del hidrogeno verde?"
results = collection.query(
    query_texts=[query],
    n_results=5  # Ajusta según cuántos resultados quieras obtener
)

print(results)

{'ids': [['noticia_23', 'noticia_74', 'noticia_147', 'noticia_379', 'noticia_212']], 'embeddings': None, 'documents': [['ENAP EXPONE AVANCES DE PROYECTOS DE HIDRÓGENO VERDE (empty)', 'Hidrógeno verde: Países Bajos desea aportar en la infraestructura portuaria y ayudar al desarrollo de la cadena de valor – Eliana Ximena Oyarzún Nancuante “Hace algunos años, hemos designado la transición energética como prioridad nacional, ligada a nuestro deber y cumplimiento del Acuerdo de París y la reducción de emisiones y con el hecho de que está cada vez más claro que es necesario diversificar las fuentes de energía”. Así lo planteó la embajadora de los Países Bajos, Carmen Gonsalves, al momento de conversar sobre las potencialidades que tiene para tales fines el desarrollo de e-combustibles y el interés de su gobierno y un conjunto importante de empresas privadas y semiestatales en poder invertir en nuestra región para apoyar el desarrollo de la industria del hidrógeno verde en Magallanes. De hech

In [ ]:
import openai

# Inicializamos la API de OpenAI (ChatGPT)
openai.api_key = 'api-api-6j2dFP7NTXvKdFOf3n1j8A'


In [7]:
# Extraer el texto de los documentos devueltos
documents = results['documents']  # Asegúrate de revisar cómo es la estructura real de los resultados

# Vamos a verificar el contenido de 'documents' para asegurarnos de su estructura
print(documents)

# Si es una lista de listas, extraemos los textos directamente
texts = [doc for sublist in documents for doc in sublist]  # Aplanamos la lista en caso de ser anidada

# Imprimimos algunos documentos para revisar que todo esté correcto
print(texts[:5])

[['ENAP EXPONE AVANCES DE PROYECTOS DE HIDRÓGENO VERDE (empty)', 'Hidrógeno verde: Países Bajos desea aportar en la infraestructura portuaria y ayudar al desarrollo de la cadena de valor – Eliana Ximena Oyarzún Nancuante “Hace algunos años, hemos designado la transición energética como prioridad nacional, ligada a nuestro deber y cumplimiento del Acuerdo de París y la reducción de emisiones y con el hecho de que está cada vez más claro que es necesario diversificar las fuentes de energía”. Así lo planteó la embajadora de los Países Bajos, Carmen Gonsalves, al momento de conversar sobre las potencialidades que tiene para tales fines el desarrollo de e-combustibles y el interés de su gobierno y un conjunto importante de empresas privadas y semiestatales en poder invertir en nuestra región para apoyar el desarrollo de la industria del hidrógeno verde en Magallanes. De hecho, esta noche llegará a Punta Arenas la tercera misión público-privada de los Países Bajos para auscultar el estatus d

In [8]:
# Concatenamos los textos extraídos en un solo string
text = "\n".join(texts[:5])  # Tomamos solo los primeros 5 para simplificar

In [9]:
text

'ENAP EXPONE AVANCES DE PROYECTOS DE HIDRÓGENO VERDE (empty)\nHidrógeno verde: Países Bajos desea aportar en la infraestructura portuaria y ayudar al desarrollo de la cadena de valor – Eliana Ximena Oyarzún Nancuante “Hace algunos años, hemos designado la transición energética como prioridad nacional, ligada a nuestro deber y cumplimiento del Acuerdo de París y la reducción de emisiones y con el hecho de que está cada vez más claro que es necesario diversificar las fuentes de energía”. Así lo planteó la embajadora de los Países Bajos, Carmen Gonsalves, al momento de conversar sobre las potencialidades que tiene para tales fines el desarrollo de e-combustibles y el interés de su gobierno y un conjunto importante de empresas privadas y semiestatales en poder invertir en nuestra región para apoyar el desarrollo de la industria del hidrógeno verde en Magallanes. De hecho, esta noche llegará a Punta Arenas la tercera misión público-privada de los Países Bajos para auscultar el estatus del d

In [ ]:
from openai import OpenAI
# Configuramos el cliente de OpenAI con la clave API
chatgpt = OpenAI(
    api_key="api-api-NM5yQ9AyrOsYL2x4aR9M9qumCdzGEr01xsBFhPCR-Bn4h5M-rRD8-R6zZNUR3YA"
    
    #api_key="apiapi-v_HX4u44Jdu_haB_bq7-1mwnT-DOoYA"
)


In [11]:
# Definimos la pregunta que queremos hacer
question = query

# Creamos el prompt combinando la pregunta y los textos
prompt = f"A partir de los siguientes documentos, responde a la pregunta: {question}. \n\n{text}"

# Hacemos la consulta a ChatGPT
chat_completion = chatgpt.chat.completions.create(
    messages=[
        {
            "role": "user",


            "content": prompt,  # Aquí enviamos el prompt modificado
        }
    ],
    model="gpt-3.5-turbo",  # Modelo que quieres usar
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************R3YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# Imprimimos la respuesta generada por ChatGPT
print(chat_completion.choices[0].message.content)

# TAREA 


In [ ]:
api_key = "api-api-NM5yQ9AyrOsYL2x4aR9M9qumCdzGEr01xsBFhPCR-Bn4h5M-rRD8-R6zZNUR3YA"
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Función para almacenar documentos en ChromaDB
def store_in_chromadb(documents):
    for idx, doc in enumerate(documents):
        embedding = embeddings.embed_query(doc)
        collection.add(
            ids=[str(idx)],  # Generamos un id único para cada documento (en este caso, el índice como string)
            documents=[doc],
            embeddings=[embedding]
        )

import uuid
# Función para almacenar documentos en ChromaDB
def store_in_chromadb(documents):
    for doc in documents:
        # Generamos un UUID único para cada documento
        unique_id = str(uuid.uuid4())
        
        # Obtener el embedding para el documento
        embedding = embeddings.embed_query(doc)
        
        # Agregar el documento a la base de datos
        collection.add(
            ids=[unique_id],  # Usamos el UUID como ID
            documents=[doc],
            embeddings=[embedding]
        )

In [ ]:
# Función para realizar la búsqueda en ChromaDB
def search_in_chromadb(query):
    # Realiza la búsqueda en ChromaDB utilizando la consulta
    results = collection.query(
        query_embeddings=embeddings.embed_query(query),
        n_results=10  # Número de resultados a obtener
    )
    return results

In [ ]:
# Función para generar una respuesta usando contexto de ChromaDB
llm = ChatOpenAI(temperature=0.7, openai_api_key=api_key, model='gpt-3.5-turbo')
chain = prompt | llm
def generate_answer_with_context(query):
    # Paso 1: Buscar los documentos relevantes en ChromaDB
    search_results = search_in_chromadb(query)
    
    # Imprimir el contenido de search_results para depuración
    print("Resultados de la búsqueda:", search_results)

    # Paso 2: Asegurarnos de que los documentos son cadenas y concatenarlos como contexto
    # Revisamos el contenido de los documentos antes de concatenar
    relevant_docs = "\n".join([str(doc) for doc in search_results['documents']])
    
    # Paso 3: Usar el modelo LLM para generar la respuesta utilizando ese contexto
    prompt_template = f"Usa el siguiente contexto para responder la siguiente pregunta:\n{relevant_docs}\n\nPregunta: {query}"
    
    # Ejecutar el modelo para generar la respuesta
    answer = chain.invoke({"persona": prompt_template})
    return answer

In [ ]:
####################
####################
#buscar noticias que hablen de la tematica
def search_news_by_theme_and_dates(theme, start_date, end_date):
    query = f"{theme}"
    search_results = search_in_chromadb(query)
    
    # Filtrar las noticias por fecha
    filtered_news = []
    for doc, metadata in zip(search_results['documents'], search_results['metadatas']):
        doc_date = metadata['date']
        if start_date <= doc_date <= end_date:
            filtered_news.append(doc)
    
    return filtered_news
####################
####################
import re
from collections import Counter

def count_people_in_news(news_list):
    people_pattern = re.compile(r'\b[A-Z][a-z]*\s[A-Z][a-z]*\b')
    all_people = []

    for news in news_list:
        matches = people_pattern.findall(news)
        all_people.extend(matches)

    people_count = Counter(all_people)
    return people_count
####################
####################
#funcion para almacenar personas en ChromaDB
def filter_news_by_person(person_name):
    query = f"Menciona a {person_name}"
    search_results = search_in_chromadb(query)
    
    # Filtrar las noticias que mencionan a la persona
    filtered_news = [doc for doc in search_results['documents'] if person_name in doc]
    
    return filtered_news
####################
####################
#funcion para extraer datos personales de una persona
import re

def extract_personal_data(person_name, documents):
    personal_data = {}
    
    # Definir patrones de regex para extraer datos personales
    patterns = {
        "nombre": re.compile(rf"{person_name}"),
        "nacimiento": re.compile(r"nació en ([\w\s]+)"),
        "educación": re.compile(r"estudió en ([\w\s]+)"),
        "trabajo": re.compile(r"trabajado en ([\w\s]+)")
    }
    
    for doc in documents:
        for key, pattern in patterns.items():
            match = pattern.search(doc)
            if match:
                personal_data[key] = match.group(1)
    
    return personal_data
####################
####################
# funcion para buscar, filtrar y extraer datos personales
def get_personal_data(person_name):
    # Filtrar las noticias que mencionan a la persona
    filtered_news = filter_news_by_person(person_name)
    
    # Extraer datos personales de las noticias filtradas
    personal_data = extract_personal_data(person_name, filtered_news)
    
    return personal_data
####################
####################
import requests
def search_image(query):
    response = requests.get(f"https://api.duckduckgo.com/", params={
        "q": query,
        "format": "json",
        "no_html": 1,
        "no_redirect": 1,
        "t": "langchain"
    })
    results = response.json().get('RelatedTopics', [])
    for result in results:
        if 'Icon' in result and 'URL' in result['Icon']:
            return result['Icon']['URL']
    return "No se encontró una imagen."
####################
####################
def get_image_link(person_name):
    query = f"{person_name} foto"
    image_link = search_image(query)
    return image_link
####################
####################


In [ ]:
# Función para búsqueda en ChromaDB (o cualquier otra base de datos)
def search_tool(query):
    search_results = search_in_chromadb(query)
    return "\n".join([str(doc) for doc in search_results['documents']])

# Función para búsqueda en Wikipedia
def wikipedia_search(query):
    response = requests.get(f"https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json()['query']['pages']
    page = next(iter(pages.values()))
    return page.get('extract', "No se encontró información en Wikipedia.")

######################################################
######################################################
#Función para búsqueda en un motor de búsqueda (ejemplo usando DuckDuckGo)
def search_engine(query):
    response = requests.get(f"https://api.duckduckgo.com/", params={
        "q": query,
        "format": "json"
    })
    results = response.json().get('RelatedTopics', [])
    return "\n".join([result['Text'] for result in results if 'Text' in result])
######################################################
######################################################
# Función para extraer datos personales de una persona desde Wikipedia
def extract_personal_data_from_wikipedia(person_name):
    summary = wikipedia_search(person_name)
    personal_data = {
        "nombre": person_name,
        "resumen": summary
    }
    return personal_data
####################
####################
# Función para extraer datos personales de una persona desde un motor de búsqueda
def extract_personal_data_from_search_engine(person_name):
    summary = search_engine(person_name)
    personal_data = {
        "nombre": person_name,
        "resumen": summary
    }
    return personal_data
####################
####################

# Función para análisis de sentimientos usando TextBlob
def sentiment_analysis_tool(query):
    blob = TextBlob(query)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "El sentimiento es positivo."
    elif polarity < 0:
        return "El sentimiento es negativo."
    else:
        return "El sentimiento es neutral."

# Función para resolver cálculos matemáticos
def math_tool(query):
    try:
        # Usando eval como ejemplo, puedes integrar con algo como WolframAlpha
        result = eval(query)
        return f"El resultado del cálculo es: {result}"
    except Exception as e:
        return f"No pude resolver el cálculo: {e}"


# Inicializar el LLM
llm = OpenAI(api_key=api_key, temperature=0.7)

# Crear herramientas
tools = [
    Tool(
        name="ChromaDB Search",
        func=search_tool,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    ),
    ######################################################
    ######################################################
    Tool(
        name="Search Engine",
        func=search_engine,
        description="Realiza una búsqueda en un motor de búsqueda para obtener información relevante."
    ),
    ######################################################
    ######################################################
    Tool(
        name="Sentiment Analysis",
        func=sentiment_analysis_tool,
        description="Analiza el sentimiento de un texto."
    ),
    Tool(
         name="Math Tool",
         func=math_tool,
         description="Resuelve cálculos matemáticos simples, por ejemplo, 2+2."
    )
]

# Definir el prompt usando PromptTemplate
prompt_template = """
Eres un agente que puede interactuar con herramientas para obtener información relevante para responder preguntas. 
Tienes acceso a dos herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.

3. **Search Engine**: Realiza una búsqueda en un motor de búsqueda para obtener información relevante.

4. **Sentiment Analysis**: Analiza el sentimiento de un texto (positivo, negativo, neutral).
5. **Math Tool**: Resuelve cálculos matemáticos.

Cuando recibas una pregunta, debes decidir qué herramienta usar y cómo responder de manera concisa y precisa. 
Si es posible, utiliza primero la búsqueda en ChromaDB, si no encuentras suficiente información, consulta Wikipedia o el motor de búsqueda.Deberias utilizar ChromaDB primero.
Si la pregunta es sobre emociones o sentimientos, usa el análisis de sentimientos. 
Si la pregunta involucra matemáticas, usa la herramienta de cálculos.

Asegúrate de considerar el contexto y usar las herramientas de manera inteligente.
"""

# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)

# Crear el agente React usando `initialize_agent` (asegurando que pasemos los argumentos correctamente)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# pregunta 1

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "¿Cuales son las personas que más se mencionan entre dos fechas sobre la temática del hidrogeno verde?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

# pregunta 2

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "¿Quién es “Juan Carlos Jobet”?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

person_name = "Juan Carlos Jobet"
filtered_news = filter_news_by_person(person_name)
print("Noticias que mencionan a", person_name, ":", filtered_news)

person_name = "Juan Carlos Jobet"
personal_data = get_personal_data(person_name)
print("Datos personales de", person_name, ":", personal_data)

# pregunta 3

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "¿Cuál su opinión o su posición sobre el hidrógeno verde?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

# pregunta 4

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "¿Qué déclaraciones ha realizado en el contexto del hidrógeno verde?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

# pregunta 5 

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "Dame el enlace de una foto de Juan Carlos Jobet"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)

person_name = "Juan Carlos Jobet"
image_link = get_image_link(person_name)
print("Enlace de la foto de", person_name, ":", image_link)

# pregunta 6

In [ ]:
# Ejecutar el agente con una consulta compleja
query = "¿Quienes son las personas más mencionadas en las noticias entre dos fechas?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)